# 실습 6: Agent Runtime 배포

## 목표
- bedrock_agent_core.py 생성
- requirements.txt 생성
- Bedrock Agent Core Runtime 배포
- S3 Vectors 권한 추가

## 6-1. 환경 변수 로드

In [ ]:
from dotenv import load_dotenv
import os

load_dotenv()

REGION = os.getenv('AWS_REGION', 'us-west-2')

print(f"🌍 Region: {REGION}")

## 6-2. bedrock_agent_core.py 생성

In [ ]:
entrypoint_content = '''"""Bedrock AgentCore Runtime Entrypoint"""

from bedrock_agentcore.runtime import BedrockAgentCoreApp
from agent.factory import create_agent

app = BedrockAgentCoreApp()

@app.entrypoint
async def integrated_disaster_agent(payload):
    """Handler for agent invocation"""
    userMessage = payload.get("prompt", "Hello! How can I assist you?")
    actorId = payload.get("actor_id", "default-user1")
    sessionId = payload.get("session_id", "default-session1")
    
    print(f"Processing request - actor: {actorId}, session: {sessionId}")
    
    agent = create_agent(actorId, sessionId)
    
    stream = agent.stream_async(userMessage)
    async for event in stream:
        yield event

if __name__ == "__main__":
    app.run()
'''

with open('agent/deploy/runtime/bedrock_agent_core.py', 'w', encoding='utf-8') as f:
    f.write(entrypoint_content)

print("✅ bedrock_agent_core.py 생성 완료")

## 6-3. requirements.txt 생성

In [ ]:
requirements_content = '''strands-agents>=0.5.0
strands-agents-tools>=0.1.0
boto3>=1.34.0
bedrock-agentcore-starter-toolkit
bedrock-agentcore
wikipedia
playwright>=1.40.0
nest-asyncio
'''

with open('agent/deploy/runtime/requirements.txt', 'w') as f:
    f.write(requirements_content)

print("✅ requirements.txt 생성 완료")

## 6-4. Runtime 배포

In [ ]:
import os
os.chdir('agent/deploy/runtime')
print(f"📂 현재 디렉토리: {os.getcwd()}")

In [ ]:
from bedrock_agentcore_starter_toolkit import Runtime

print("🚀 Agent Runtime 배포 시작...\n")

agentcore_runtime = Runtime()
agent_name = "agent_runtime"

print("1️⃣ Runtime 설정 중...")
response = agentcore_runtime.configure(
    entrypoint="bedrock_agent_core.py",
    auto_create_execution_role=True,
    auto_create_ecr=True,
    requirements_file="requirements.txt",
    region=REGION,
    agent_name=agent_name
)

print("✅ Runtime 설정 완료")
print("\n2️⃣ Docker 이미지 빌드 및 배포 중... (5-10분 소요)\n")

launch_result = agentcore_runtime.launch(auto_update_on_conflict=True)

print("\n✅ 배포 완료!")
print(f"\n📝 Agent ARN: {launch_result.agent_arn}")
print(f"📝 ECR URI: {launch_result.ecr_uri}")

AGENT_RUNTIME_ARN = launch_result.agent_arn

## 6-5. Agent ARN을 .env에 저장

In [ ]:
os.chdir('../../..')

with open('.env', 'r') as f:
    lines = f.readlines()

found = False
for i, line in enumerate(lines):
    if line.startswith('AGENT_RUNTIME_ARN='):
        lines[i] = f"AGENT_RUNTIME_ARN={AGENT_RUNTIME_ARN}\n"
        found = True
        break

if not found:
    lines.append(f"AGENT_RUNTIME_ARN={AGENT_RUNTIME_ARN}\n")

with open('.env', 'w') as f:
    f.writelines(lines)

print(f"✅ Agent ARN을 .env에 저장 완료")
print(f"📝 ARN: {AGENT_RUNTIME_ARN}")

## 6-6. S3 Vectors 권한 추가

In [ ]:
%run lab4_s3vectors_permission.py

## ✅ 실습 6 완료!

### 생성된 리소스
- Agent Runtime (Bedrock Agent Core)
- ECR Repository
- Docker Image
- Execution Role (S3 Vectors 권한 포함)

### 생성된 파일
- `agent/deploy/runtime/bedrock_agent_core.py`
- `agent/deploy/runtime/requirements.txt`
- `.env` (AGENT_RUNTIME_ARN 추가)

### 다음 단계
👉 **lab7_web_application.ipynb**